In [ ]:
#  Image Classification using Keras Library

In [ ]:
!wget --no-check-certificate \
  https://dicodingacademy.blob.core.windows.net/picodiploma/ml_pemula_academy/messy-vs-clean-room.zip \
  -O messy_vs_clean_room.zip

--2021-05-11 14:12:42--  https://dicodingacademy.blob.core.windows.net/picodiploma/ml_pemula_academy/messy-vs-clean-room.zip
Resolving dicodingacademy.blob.core.windows.net (dicodingacademy.blob.core.windows.net)... 52.239.197.36
Connecting to dicodingacademy.blob.core.windows.net (dicodingacademy.blob.core.windows.net)|52.239.197.36|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 70392746 (67M) [application/zip]
Saving to: ‘messy_vs_clean_room.zip’

messy_vs_clean_room 100%[===================>]  67.13M  4.29MB/s    in 15s     

2021-05-11 14:12:58 (4.61 MB/s) - ‘messy_vs_clean_room.zip’ saved [70392746/70392746]



In [ ]:
import zipfile,os

In [ ]:
local_zip = 'messy_vs_clean_room.zip'

In [ ]:
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

In [ ]:
base_dir = '/tmp/images'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'val')

In [ ]:
import tensorflow as tf

In [ ]:
# membuat direktori ruangan rapi pada direktori data training
train_kotor_dir = os.path.join(train_dir, 'bersih')

In [ ]:
# membuat direktori ruangan berantakan pada direktori data training
train_bersih_dir = os.path.join(train_dir, 'kotor')

In [ ]:
# membuat direktori ruangan rapi pada direktori data validasi
validation_bersih_dir = os.path.join(validation_dir, 'bersih')

In [ ]:
# membuat direktori ruangan berantakan pada direktori data validasi
c = os.path.join(validation_dir, 'kotor')

In [ ]:
#Langkah selanjutnya adalah kita membuat sebuah objek ImageDataGenerator untuk data training dan data testing
"""

Image data generator adalah sebuah fungsi yang sangat berguna untuk mempersiapkan data latih dan data testing yang akan diberikan ke model. 
Beberapa kemudahan yang disediakan Image data generator adalah, preprocessing data, pelabelan sampel otomatis, dan augmentasi gambar.

"""

'\n\nImage data generator adalah sebuah fungsi yang sangat berguna untuk mempersiapkan data latih dan data testing yang akan diberikan ke model. \nBeberapa kemudahan yang disediakan Image data generator adalah, preprocessing data, pelabelan sampel otomatis, dan augmentasi gambar.\n\n'

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [ ]:
train_datagen = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=20,
                    horizontal_flip=True,
                    shear_range = 0.2,
                    fill_mode = 'nearest')
 
test_datagen = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=20,
                    horizontal_flip=True,
                    shear_range = 0.2,
                    fill_mode = 'nearest')

In [ ]:
# Lalu kita dapat menggunakan objek image data generator sebelumnya untuk mempersiapkan data latih yang akan dipelajari oleh model.

In [ ]:
train_generator = train_datagen.flow_from_directory(
        train_dir,  # direktori data latih
        target_size=(150, 150),  # mengubah resolusi seluruh gambar menjadi 150x150 piksel
        batch_size=32,
        # karena kita merupakan masalah klasifikasi 2 kelas maka menggunakan class_mode = 'binary'
        class_mode='binary')
 
validation_generator = test_datagen.flow_from_directory(
        validation_dir, # direktori data validasi
        target_size=(150, 150), # mengubah resolusi seluruh gambar menjadi 150x150 piksel
        batch_size=32, # karena kita merupakan masalah klasifikasi 2 kelas maka menggunakan class_mode = 'binary'
        class_mode='binary')

Found 192 images belonging to 2 classes.
Found 20 images belonging to 2 classes.


In [ ]:
# Setelah data telah siap, kita bisa membangun arsitektur sebuah CNN

In [ ]:
#  Anda tentu masih ingat bahwa fungsi dari layer konvolusi adalah untuk mengekstraksi atribut pada gambar. 
#  Sedangkan layer max pooling berguna untuk mereduksi resolusi gambar sehingga proses pelatihan MLP lebih cepat.

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)), # 32 filter, (3,3) kernelnya, aktifatornya menggunakan relu input demensi 150x150x3 (3 untuk rgb)
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu', ),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
# compile model dengan 'adam' optimizer loss function 'binary_crossentropy' 
model.compile(loss='binary_crossentropy',
              optimizer=tf.optimizers.Adam(),
              metrics=['accuracy'])

In [ ]:
 """ Setelah menentukan loss function dan optimizer pada CNN, kita dapat melatih model kita menggunakan metode fit. 
 Dengan menggunakan image data generator, kita tidak perlu memasukkan parameter gambar dan labelnya. Image data generator secara otomatis melabeli 
 sebuah gambar sesuai dengan direktori di mana ia disimpan. 
 Contohnya sebuah gambar yang terdapat di direktori clean, secara otomatis akan diberi label “clean” oleh image data generator.

In [ ]:
model.fit(
      train_generator,
      steps_per_epoch=25,  # berapa batch yang akan dieksekusi pada setiap epoch
      epochs=100, #  iteration over the entire x and y data provided
      validation_data=validation_generator, # menampilkan akurasi pengujian data validasi
      validation_steps=5,  # berapa batch yang akan dieksekusi pada setiap epoch
      verbose=2)

Epoch 1/100
25/25 - 8s - loss: 0.7234 - accuracy: 0.4836 - val_loss: 0.6924 - val_accuracy: 0.6000
Epoch 2/100
25/25 - 7s - loss: 0.7014 - accuracy: 0.5840
Epoch 3/100
25/25 - 7s - loss: 0.6876 - accuracy: 0.5410
Epoch 4/100
25/25 - 7s - loss: 0.6881 - accuracy: 0.4960
Epoch 5/100
25/25 - 7s - loss: 0.6701 - accuracy: 0.6148
Epoch 6/100
25/25 - 7s - loss: 0.6708 - accuracy: 0.5820
Epoch 7/100
25/25 - 7s - loss: 0.6056 - accuracy: 0.7120
Epoch 8/100
25/25 - 7s - loss: 0.6854 - accuracy: 0.6148
Epoch 9/100
25/25 - 7s - loss: 0.6164 - accuracy: 0.7049
Epoch 10/100
25/25 - 7s - loss: 0.5616 - accuracy: 0.7377
Epoch 11/100
25/25 - 7s - loss: 0.6258 - accuracy: 0.7049
Epoch 12/100
25/25 - 7s - loss: 0.5881 - accuracy: 0.7049
Epoch 13/100
25/25 - 7s - loss: 0.6094 - accuracy: 0.6885
Epoch 14/100
25/25 - 7s - loss: 0.5613 - accuracy: 0.7541
Epoch 15/100
25/25 - 7s - loss: 0.5031 - accuracy: 0.7951
Epoch 16/100
25/25 - 7s - loss: 0.5465 - accuracy: 0.7200
Epoch 17/100
25/25 - 7s - loss: 0.4884 

In [ ]:
#import numpy as np
#from google.colab import files
from keras.preprocessing import image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
 
img = image.load_img('9.png', target_size=(150,150))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
 
images = np.vstack([x])
classes = model.predict(images, batch_size=10)
print(classes)
if classes == 0:
  print('kamar bersih')
else:
  print('kamar kotor')

[[1.]]
kamar kotor


**diatas prediksi bahwa kamar KOTOR**

In [ ]:
image = tf.zeros([5,5,3])
tf.expand_dims(image, axis=0)

In [ ]:
classifier.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))

"""parameter pertama (32) adalah nilai filter nya. parameter kedua adalah dimensi filter nya (3×3), parameter ketiga cukup self 
    explained disini yakni bentuk input nya dan tipe nya (RGB atau Black/white), 
    parameter keempat adalah fungsi aktivasi yang kita gunakan (relu) disini kita menggunakan rectifier function.